In [1]:
import pandas as pd

import requests 

import os

import json

import googleapiclient.discovery
import googleapiclient.errors

import numpy as np

In [2]:
chiave_yt = json.load(open('key.json', 'r'))['youtube']['key'][1]
youtube = googleapiclient.discovery.build('youtube', 'v3', developerKey=chiave_yt)

In [25]:
quota

1581

In [3]:
#insert initial quota value
#MODIFICA MAXQUOTA A 9900!!
quota = 0

In [4]:
df_videos = pd.read_csv('data/get_youtuber_videos_output/youtuber_videos.csv')

In [5]:
def get_infos(response):
    info_comment_list=[]
    video_id=next_page_token = thread_comment_id = top_level_comment_id = author_display_name = author_channel_id = author_channel_id = text_original=None
    try:
        next_page_token = response['nextPageToken']
    except KeyError as e:
        #print(response)
        next_page_token = None
        print('Page token non esistente')
    items = response['items']
    for item in items:
        thread_comment_id = item['id']
        try:
            video_id = item['snippet']['videoId']
            top_level_comment = item['snippet']['topLevelComment']
            top_level_comment_id = top_level_comment['id']
            top_level_comment_snippet = top_level_comment['snippet']
            author_display_name = top_level_comment_snippet['authorDisplayName']
            text_original = top_level_comment_snippet['textOriginal']
            author_channel_id = top_level_comment_snippet['authorChannelId']['value']
        except KeyError as e:
            print(e.args[0])
            print('Problem with commentThread {}'.format(thread_comment_id))
        info_comment_list.append([video_id,thread_comment_id,top_level_comment_id,author_display_name,author_channel_id,text_original])
    return next_page_token, info_comment_list

In [6]:
#############################################################################

In [7]:
#round 1
#la prossima volta togli e decommenta quello sotto

In [8]:
last_csv = int(np.load('data/get_comment_threads_output/last_csv.npy'))
next_page_token = str(np.load('data/get_comment_threads_output/next_page_token.npy'))
last_video_idx = int(np.load('data/get_comment_threads_output/video_to_start.npy'))
videos_not_finished = (np.load('data/get_comment_threads_output/videos_not_finished.npy')).tolist()
videos_disabled_comments = (np.load('data/get_comment_threads_output/videos_disabled_comments.npy')).tolist()
print("Starting with video #{}, next_page_token: {}".format(last_video_idx,next_page_token))
print("Writing on {} csv".format(last_csv+1))

Starting with video #2399, next_page_token: QURTSl9pMGRlWWU5S2k3SzZsR1VoaUdad2FmejFMYl8xMC1sa3dWRk5uRnZJUlpPODdGcW5HQ3U2a1h2MllRVFZIbzFJM3oyWVFHcE5mZ3NkY0tqWXlRMTFTb1V2RHhzREtWbkRXb25IWEpSWUhRWU52MmF0NkM0eDMtUmhoNDNJb200bGxJMXgtN3RDdkJoSGRJdFZ3RmxmS3BMOUxUbWlRR0NnOV9faEtUVQ==
Writing on 5 csv


In [9]:
df_videos_0 = df_videos[last_video_idx:]

In [10]:
#per i video con più di 10000 commenti, salvo l'indice del video con il next page token

In [11]:
comments = []
comments_count = 0

In [12]:
df_videos_0

channel_id     video_id
2399  UCOtoxvHLKrIlWbt4MRBWfbQ  lSlKqgyE2Zo
2400  UCOtoxvHLKrIlWbt4MRBWfbQ  _5ki0E6arN0
2401  UCOtoxvHLKrIlWbt4MRBWfbQ  5ac6JqEeD1U
2402  UCOtoxvHLKrIlWbt4MRBWfbQ  xa7XZF4wA70
2403  UCOtoxvHLKrIlWbt4MRBWfbQ  0ztsAOrPdzQ
...                        ...          ...
2588  UCq7EY7H2XF6TV7Z5mLv-aNg  bm-3Qrr7EUA
2589  UCq7EY7H2XF6TV7Z5mLv-aNg  hkxRjDS_Cjw
2590  UCq7EY7H2XF6TV7Z5mLv-aNg  8kElrRAa1ws
2591  UCq7EY7H2XF6TV7Z5mLv-aNg  7x6tX5T7LOs
2592  UCq7EY7H2XF6TV7Z5mLv-aNg  NSdUFKOkKnc

[194 rows x 2 columns]

In [13]:
for video in df_videos_0['video_id'].values:
    print('________________________________________________________')
    print('{} video: {}'.format(last_video_idx, video))
    video_comments_count = 0
    while next_page_token != None and video_comments_count<10000:
        print('Reached quotas: {}'.format(quota))
        if quota > 9900:
            print('Quota exceeded')
            break
        else:
            request = youtube.commentThreads().list(
                part="snippet",
                maxResults=100,
                pageToken=next_page_token,
                videoId=video
            )
            try:
                response = request.execute()
                quota += 3
                next_page_token, results = get_infos(response)
                video_comments_count+=len(results)
                comments_count+=video_comments_count
                comments.append(results)
            except:
                videos_disabled_comments.append(video)
                next_page_token = None
                print('Video has disabled comments')
    if quota > 9900:
        break
    if next_page_token == None or video_comments_count>=10000:
        if next_page_token != None:
            print('Reached max comment count')
            videos_not_finished.append([video, next_page_token])
        last_video_idx += 1
        next_page_token = ' '
        print('Total comments for {}: {}'.format(video, video_comments_count))

print("Done with {} comments, video to start: {}\n with next_page_token: {}".format(comments_count,last_video_idx, next_page_token))

________________________________________________________
2399 video: lSlKqgyE2Zo
Reached quotas: 0
Reached quotas: 3
Reached quotas: 6
Reached quotas: 9
Reached quotas: 12
Reached quotas: 15
Reached quotas: 18
Reached quotas: 21
Reached quotas: 24
Reached quotas: 27
Reached quotas: 30
Reached quotas: 33
Reached quotas: 36
Reached quotas: 39
Reached quotas: 42
Reached quotas: 45
Reached quotas: 48
Reached quotas: 51
Reached quotas: 54
Reached quotas: 57
Reached quotas: 60
Reached quotas: 63
Reached quotas: 66
Reached quotas: 69
Reached quotas: 72
Reached quotas: 75
Reached quotas: 78
Reached quotas: 81
Reached quotas: 84
Reached quotas: 87
Reached quotas: 90
Reached quotas: 93
Reached quotas: 96
Reached quotas: 99
Reached quotas: 102
Reached quotas: 105
Reached quotas: 108
Reached quotas: 111
Reached quotas: 114
Reached quotas: 117
Reached quotas: 120
Reached quotas: 123
Reached quotas: 126
Reached quotas: 129
Reached quotas: 132
Reached quotas: 135
Page token non esistente
Total commen

Reached quotas: 702
Page token non esistente
Total comments for mjmB13AKBjU: 680
________________________________________________________
2424 video: jXhEhtNVsq4
Reached quotas: 705
Reached quotas: 708
Page token non esistente
Total comments for jXhEhtNVsq4: 119
________________________________________________________
2425 video: 0BjHKhehvog
Reached quotas: 711
Reached quotas: 714
Page token non esistente
Total comments for 0BjHKhehvog: 127
________________________________________________________
2426 video: fWb9QyiO-YU
Reached quotas: 717
Reached quotas: 720
Page token non esistente
Total comments for fWb9QyiO-YU: 115
________________________________________________________
2427 video: D0xY9MZNwaw
Reached quotas: 723
Page token non esistente
Total comments for D0xY9MZNwaw: 94
________________________________________________________
2428 video: V-L9NwXX8cI
Reached quotas: 726
Reached quotas: 729
Page token non esistente
Total comments for V-L9NwXX8cI: 173
______________________________

Reached quotas: 1032
Reached quotas: 1035
Page token non esistente
Total comments for pZrom_dUZZc: 242
________________________________________________________
2466 video: zU5V8D_e5xE
Reached quotas: 1038
Reached quotas: 1041
Page token non esistente
Total comments for zU5V8D_e5xE: 172
________________________________________________________
2467 video: WmRGtjBAdLU
Reached quotas: 1044
Reached quotas: 1047
Page token non esistente
Total comments for WmRGtjBAdLU: 129
________________________________________________________
2468 video: AoJ3ORjvoug
Reached quotas: 1050
Page token non esistente
Total comments for AoJ3ORjvoug: 54
________________________________________________________
2469 video: _21x9V6kjKk
Reached quotas: 1053
Reached quotas: 1056
Reached quotas: 1059
Reached quotas: 1062
Reached quotas: 1065
Page token non esistente
Total comments for _21x9V6kjKk: 488
________________________________________________________
2470 video: _Mf7EnT0tsY
Reached quotas: 1068
Reached quotas: 10

Page token non esistente
Total comments for -hxYnqqH46I: 34
________________________________________________________
2512 video: CUMbr_Fkl5o
Reached quotas: 1272
Page token non esistente
Total comments for CUMbr_Fkl5o: 31
________________________________________________________
2513 video: MniwsP5bcXc
Reached quotas: 1275
Page token non esistente
Total comments for MniwsP5bcXc: 53
________________________________________________________
2514 video: VqNrKn55MJI
Reached quotas: 1278
Page token non esistente
Total comments for VqNrKn55MJI: 61
________________________________________________________
2515 video: 9hAuMdcQX_Y
Reached quotas: 1281
Reached quotas: 1284
Page token non esistente
Total comments for 9hAuMdcQX_Y: 118
________________________________________________________
2516 video: 8ragvJFG3i4
Reached quotas: 1287
Page token non esistente
Total comments for 8ragvJFG3i4: 42
________________________________________________________
2517 video: bsuIiWsAhW4
Reached quotas: 1290
Page t

Page token non esistente
Total comments for HQSgZEanrhA: 88
________________________________________________________
2560 video: KOsMswGKMu8
Reached quotas: 1476
Page token non esistente
authorChannelId
Problem with commentThread UgwRmTbunOef6zhiq-F4AaABAg
Total comments for KOsMswGKMu8: 10
________________________________________________________
2561 video: 1miNGepOsbs
Reached quotas: 1479
Page token non esistente
Total comments for 1miNGepOsbs: 14
________________________________________________________
2562 video: AzJpJQTrIKc
Reached quotas: 1482
Page token non esistente
Total comments for AzJpJQTrIKc: 4
________________________________________________________
2563 video: Q-fOV81_Y0s
Reached quotas: 1485
Page token non esistente
Total comments for Q-fOV81_Y0s: 1
________________________________________________________
2564 video: dCQmPL3FISc
Reached quotas: 1488
Page token non esistente
Total comments for dCQmPL3FISc: 20
________________________________________________________
2565 

In [14]:
np.save('data/get_comment_threads_output/next_page_token.npy',next_page_token)
np.save('data/get_comment_threads_output/video_to_start.npy',last_video_idx)
np.save('data/get_comment_threads_output/videos_not_finished.npy',videos_not_finished)
np.save('data/get_comment_threads_output/videos_disabled_comments.npy',videos_disabled_comments)

In [15]:
print(next_page_token, last_video_idx)

  2593


In [16]:
format_ok_comments = []
for outer_comment in comments:
    for comment in outer_comment:
        format_ok_comments.append(comment)

In [17]:
format_ok_comments

[['lSlKqgyE2Zo',
  'UgyBkSueeWLhRQ8-FyR4AaABAg',
  'UgyBkSueeWLhRQ8-FyR4AaABAg',
  'luckydolly',
  'UC4BuW8UZuS4WTjDtD3aY17A',
  'I have had the same experience with a doctor. And so has my mom. She was like 6-7 months pregnant and she started bleeding. She went to the hospital and the doctor came in and just looked at her and said well you lost the baby ok. Then just walked out. Doctors like this shouldn’t even be allowed to see patients or have a license.'],
 ['lSlKqgyE2Zo',
  'Ugz6AQBHHhAtX9cdPnR4AaABAg',
  'Ugz6AQBHHhAtX9cdPnR4AaABAg',
  'Emory Elliott',
  'UCXq5M8DVAQlX55rZ-1HqREQ',
  "The negative bias is what you're talking about towards the end, there. Our brains are wired to remember and hold on to the bad things that happen instead of the good. \n\nSo sorry about that shit doctor, that was extremely unprofessional and insensitive. I hope you feel better, and fuck that guy."],
 ['lSlKqgyE2Zo',
  'UgzmrA6R9UuHBIYYmWV4AaABAg',
  'UgzmrA6R9UuHBIYYmWV4AaABAg',
  'Rede4trip',
  'UC

In [18]:
df_comment_thread = pd.DataFrame(format_ok_comments,columns=['video_id','comment_thread_id','top_level_comment_id','author_display_name','author_channel_id','text_original'])

In [19]:
df_comment_thread

video_id           comment_thread_id        top_level_comment_id  \
0      lSlKqgyE2Zo  UgyBkSueeWLhRQ8-FyR4AaABAg  UgyBkSueeWLhRQ8-FyR4AaABAg   
1      lSlKqgyE2Zo  Ugz6AQBHHhAtX9cdPnR4AaABAg  Ugz6AQBHHhAtX9cdPnR4AaABAg   
2      lSlKqgyE2Zo  UgzmrA6R9UuHBIYYmWV4AaABAg  UgzmrA6R9UuHBIYYmWV4AaABAg   
3      lSlKqgyE2Zo  UgzImr_PGPXs52nLoAx4AaABAg  UgzImr_PGPXs52nLoAx4AaABAg   
4      lSlKqgyE2Zo  UgxxcDuBbF7C611ooRB4AaABAg  UgxxcDuBbF7C611ooRB4AaABAg   
...            ...                         ...                         ...   
42332  8kElrRAa1ws  UgwSdanGKGsGd4DUmIl4AaABAg  UgwSdanGKGsGd4DUmIl4AaABAg   
42333  8kElrRAa1ws  Ugy-wtIUI069bRoN9K14AaABAg  Ugy-wtIUI069bRoN9K14AaABAg   
42334  8kElrRAa1ws  UgwtkQootxJ6rfJzvPh4AaABAg  UgwtkQootxJ6rfJzvPh4AaABAg   
42335  8kElrRAa1ws  Ugz_Yj_AX5darh32Y7t4AaABAg  Ugz_Yj_AX5darh32Y7t4AaABAg   
42336  8kElrRAa1ws  Ugx9T5pgiWFHmv6Glbl4AaABAg  Ugx9T5pgiWFHmv6Glbl4AaABAg   

      author_display_name         author_channel_id  \
0              luckydolly  UC4BuW8UZuS4WTjDtD3aY17A   
1           Emory Elliott  UCXq5M8DVAQlX55rZ-1HqREQ   
2               Rede4trip  UCy6dVt-_KE1UJCUr46meT_Q   
3          MARCELA Bukalo  UC_hISte6WW9yksrOAGyVyBQ   
4      saiiree yessenniia  UC8G4mgg7HlG096cG2bwWDHQ   
...                   ...                       ...   
42332              Bree e  UCxbprPK-QSbBE6Zdu8eQ7Bw   
42333         Tonya Wykle  UClgzRcV9-SJl-pTbAyAUoZw   
42334               Lesly  UCfzU2ed9ApsBbgor5BIdT1w   
42335       Scarlet Black  UC3RHBku3ppkK4rcrZkub6JA   
42336    Milkshake Nugget  UCyFb0sqS_sKWE4uPzy8PpMA   

                                           text_original  
0      I have had the same experience with a doctor. ...  
1      The negative bias is what you're talking about...  
2      I am sooooo the same way about focusing on the...  
3      I have PCOS, had fibroids surgically removed, ...  
4      WE NEED A NAME! I am so upset this happened to...  
...                                                  ...  
42332  I love this mascara for real tho. It's easy to...  
42333  Hello makeup world, nice video. Xoxo princess 🎈🎉🎊  
42334                                              Song?  
42335                    Thanos would love this mascara.  
42336                                   I like raccoons.  

[42337 rows x 6 columns]

In [20]:
last_csv

4

In [21]:
last_csv+=1
last_csv

5

In [22]:
df_comment_thread.to_csv('data/get_comment_threads_output/df_comment_thread_{}.csv'.format(last_csv), index = False)

In [26]:
np.save('data/get_comment_threads_output/last_csv.npy',last_csv)

In [24]:
df = pd.read_csv('data/get_comment_threads_output/df_comment_thread_{}.csv'.format(last_csv))
df

video_id           comment_thread_id        top_level_comment_id  \
0      lSlKqgyE2Zo  UgyBkSueeWLhRQ8-FyR4AaABAg  UgyBkSueeWLhRQ8-FyR4AaABAg   
1      lSlKqgyE2Zo  Ugz6AQBHHhAtX9cdPnR4AaABAg  Ugz6AQBHHhAtX9cdPnR4AaABAg   
2      lSlKqgyE2Zo  UgzmrA6R9UuHBIYYmWV4AaABAg  UgzmrA6R9UuHBIYYmWV4AaABAg   
3      lSlKqgyE2Zo  UgzImr_PGPXs52nLoAx4AaABAg  UgzImr_PGPXs52nLoAx4AaABAg   
4      lSlKqgyE2Zo  UgxxcDuBbF7C611ooRB4AaABAg  UgxxcDuBbF7C611ooRB4AaABAg   
...            ...                         ...                         ...   
42332  8kElrRAa1ws  UgwSdanGKGsGd4DUmIl4AaABAg  UgwSdanGKGsGd4DUmIl4AaABAg   
42333  8kElrRAa1ws  Ugy-wtIUI069bRoN9K14AaABAg  Ugy-wtIUI069bRoN9K14AaABAg   
42334  8kElrRAa1ws  UgwtkQootxJ6rfJzvPh4AaABAg  UgwtkQootxJ6rfJzvPh4AaABAg   
42335  8kElrRAa1ws  Ugz_Yj_AX5darh32Y7t4AaABAg  Ugz_Yj_AX5darh32Y7t4AaABAg   
42336  8kElrRAa1ws  Ugx9T5pgiWFHmv6Glbl4AaABAg  Ugx9T5pgiWFHmv6Glbl4AaABAg   

      author_display_name         author_channel_id  \
0              luckydolly  UC4BuW8UZuS4WTjDtD3aY17A   
1           Emory Elliott  UCXq5M8DVAQlX55rZ-1HqREQ   
2               Rede4trip  UCy6dVt-_KE1UJCUr46meT_Q   
3          MARCELA Bukalo  UC_hISte6WW9yksrOAGyVyBQ   
4      saiiree yessenniia  UC8G4mgg7HlG096cG2bwWDHQ   
...                   ...                       ...   
42332              Bree e  UCxbprPK-QSbBE6Zdu8eQ7Bw   
42333         Tonya Wykle  UClgzRcV9-SJl-pTbAyAUoZw   
42334               Lesly  UCfzU2ed9ApsBbgor5BIdT1w   
42335       Scarlet Black  UC3RHBku3ppkK4rcrZkub6JA   
42336    Milkshake Nugget  UCyFb0sqS_sKWE4uPzy8PpMA   

                                           text_original  
0      I have had the same experience with a doctor. ...  
1      The negative bias is what you're talking about...  
2      I am sooooo the same way about focusing on the...  
3      I have PCOS, had fibroids surgically removed, ...  
4      WE NEED A NAME! I am so upset this happened to...  
...                                                  ...  
42332  I love this mascara for real tho. It's easy to...  
42333  Hello makeup world, nice video. Xoxo princess 🎈🎉🎊  
42334                                              Song?  
42335                    Thanos would love this mascara.  
42336                                   I like raccoons.  

[42337 rows x 6 columns]